In [42]:
import string

def get_matrix(area: str):
    return list(map(lambda line: [int(point) if point in string.digits else -1 for point in list(line)], area.split("\n")))


def render_matrix(matrix):
    return "\n".join(["".join([str(x) if x > -1 else "." for x in line]) for line in matrix])


class Point():
    parent = None # Point()
    value = None # 0 - 9
    coordinates = None # (x, y)
    childs =  None # [Point(),]
    relatives = None
    matrix = None
    is_trailhead = False
    
    def __init__(self, matrix, coordinates, is_trailhead=False):
        x,y = coordinates
        self.value = matrix[y][x]
        self.matrix = matrix
        self.coordinates = coordinates
        self.childs = []
        self.relatives = []
        self.is_trailhead = is_trailhead

    def _bring_to_parent(self, child):
        if self.is_trailhead:
            self.relatives.append(child)
            return self
        self.parent._bring_to_parent(child)

    def add_child(self, coordinates):
        child = Point(self.matrix, [*coordinates])
        child.parent = self
        self.childs.append(child)
        self._bring_to_parent(child)
        return child


    def _get_destinations(self):
        return map(lambda child: tuple(child.coordinates), filter(lambda child: child.value == 9, self.relatives))

    def get_score(self):
        return len(set(self._get_destinations()))

    def get_trails_score(self):
        return len(tuple(self._get_destinations()))


    def __str__(self):
        x,y = self.coordinates
        return f'{{"xy": [{x}, {y}], "val": {self.value}}}'

    def visualize(self):
        # clone = get_matrix(render_matrix(self.matrix))
        clone = [[str(x) for x in line] for line in self.matrix]

        for child in self.childs:
            x,y = child.coordinates
            clone[y][x] = "\033[0;43m" + str(child.value) + "\033[0m"

        px,py = self.coordinates
        clone[py][px] = "\033[0;44m" + str(self.value) + "\033[0m"
        print("\n".join(["".join(line) for line in clone]))
        

    def __repr__(self):
        return str(self)


def get_trailheads(matrix):
    points = []
    for y in range(len(matrix)):
        for x in range(len(matrix[y])):
            if matrix[y][x] == 0:
                points.append((x, y))
    return points

def get_next_points(matrix: tuple[tuple[int]], current: tuple[int]):
    following = []
    x,y = current
    current_val = matrix[y][x]

    if current_val < 0:
        return []
    
    if y > 0:
        if matrix[y-1][x] - current_val == 1:
            following.append((x, y-1))

    if y < len(matrix) -1:
        if matrix[y+1][x] - current_val == 1:
            following.append((x, y+1))

    if x > 0:
        if matrix[y][x-1] - current_val == 1:
            following.append((x-1, y))

    if x < len(matrix[y]) -1:
        if matrix[y][x+1] - current_val == 1:
            following.append((x+1, y))

    return following

def get_trails(matrix, point):
    # print(point.coordinates)
    next_points = get_next_points(matrix, point.coordinates)
    # print(next_points)
    for x,y in next_points:
        child = point.add_child((x ,y))
        get_trails(matrix, child)
    return point

with open("input.txt", "r") as f:
    topo = f.read()

matrix = get_matrix(topo)

score = 0
for trailheads in get_trailheads(matrix):
    trailhead_point = Point(matrix, trailheads, is_trailhead=True)
    trail_tree = get_trails(matrix, trailhead_point)
    trail_score = trail_tree.get_score()
    # print(f"🔥 {trail_score=}")
    score += trail_score

score

816

Correct: `816`

In [41]:
def visualize(matrix, points):
    # clone = get_matrix(render_matrix(self.matrix))
    clone = [[str(x) if x > -1 else "." for x in line] for line in matrix]
    trailhead = points[0]
    
    for point in points:
        x,y = point.coordinates
        clone[y][x] = "\033[0;43m" + str(point.value) + "\033[0m"

    px,py = trailhead.coordinates
    clone[py][px] = "\033[0;44m" + str(trailhead.value) + "\033[0m"
    print("\n".join(["".join(line) for line in clone]))


with open("input.txt", "r") as f:
    topo = f.read()

matrix = get_matrix(topo)

score = 0
for trailheads in get_trailheads(matrix):
    trailhead_point = Point(matrix, trailheads, is_trailhead=True)
    trail_tree = get_trails(matrix, trailhead_point)
    score += trail_tree.get_trails_score()

score

1960

Correct: `1960` 